In [68]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda

In [69]:
from keras.layers.convolutional import Conv2D, Cropping2D

In [70]:
import numpy as np
import pandas as pd
import cv2

In [71]:
from glob import glob
driving_logs = glob("driving_log*.csv")

In [72]:
# Gather logs from all datasets
logs_df = [pd.read_csv(log, header=None) for log in driving_logs]

In [74]:
# Concatenate driving logs
driving_log = pd.concat(logs_df)
# Process image paths (get rid of backslashes) to get only file names 
driving_log.iloc[:, 0] = driving_log.iloc[:, 0].str.replace("\\", "/").str.split("/").str[-1]
driving_log.iloc[:, 1] = driving_log.iloc[:, 1].str.replace("\\", "/").str.split("/").str[-1]
driving_log.iloc[:, 2] = driving_log.iloc[:, 2].str.replace("\\", "/").str.split("/").str[-1]
# Feature dataset
features = driving_log.iloc[:, [0, 1, 2, 3]]
# See stats for driving telemetry. First column is steering angle
driving_log.describe()

,3,4,5,6
count,22812.000000,22812.000000,22812.000000,2.281200e+04
mean,-0.024412,0.820748,0.000478,2.925521e+01
std,0.134234,0.296372,0.013991,3.534222e+00
min,-1.000000,0.000000,0.000000,1.851782e-07
25%,-0.041394,0.670782,0.000000,3.010205e+01
50%,0.000000,1.000000,0.000000,3.018979e+01
75%,0.000000,1.000000,0.000000,3.019028e+01
max,0.845074,1.000000,0.583636,3.078679e+01


In [75]:
# Frames from central camera
X_center, y_center = features.iloc[:, 0].values, features.iloc[:, 3].values
# Frames from left camera with augmented steering angle
X_left, y_left = features.iloc[:, 1].values, features.iloc[:, 3].values + 0.07
# Frames from right camera with augmented angle
X_right, y_right = features.iloc[:, 2].values, features.iloc[:, 3].values - 0.07
# Resulting dataset
X, y = np.concatenate([X_center, X_left, X_right]), np.concatenate([y_center, y_left, y_right])

In [76]:
# Dataset has now 68436 images
X.shape, y.shape

((68436,), (68436,))

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
# Split training data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
#X_train, X_valid, y_train, y_valid = train_test_split(
#    X_train, y_train, test_size=0.2, random_state=42)

In [65]:
from sklearn.utils import shuffle

BATCH_SIZE=32 # number of samples in a batch
EPOCH_STEPS = 2*X_train.shape[0]//BATCH_SIZE # number of batches for 1 epoch
EPOCHS = 5 # number of epochs
VALID_STEPS = 2*X_test.shape[0]//BATCH_SIZE # amount of validation batches

# Data generator. 
# For each batch takes BATCH_SIZE//2 original images and the same amount
# of flipped left to right images with negative steering angles

def datagen(X, y, batch_size=BATCH_SIZE):
    num_samples = X.shape[0]
    while 1:
        shuffle(X, y)
        for offset in range(0, num_samples, batch_size//2):
            files = X[offset:offset+batch_size//2]
            angles = y[offset:offset+batch_size//2]
            image_batch = []
            angle_batch = []
            for f, a in zip(files, angles):
                image = cv2.imread('IMG/' + f)
                # Resize original image
                small = cv2.resize(image, (240, 120))
                # Convert image to YUV color space
                image = cv2.cvtColor(small, cv2.COLOR_BGR2YUV)
                image_batch.append(image)
                image_batch.append(np.fliplr(image))
                angle_batch.append(float(a))
                angle_batch.append(float(-a))
                
            yield shuffle(np.array(image_batch), np.array(angle_batch))
train_gen = datagen(X_train, y_train)
valid_gen = datagen(X_test, y_test)

In [66]:
# Model is based on Nvidia end-to-end self-driving network architecture
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(120,240,3)))
model.add(Cropping2D(cropping=((45, 15), (0, 0))))
model.add(Conv2D(24, (5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(36, (5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(48, (5,5), strides=(2,2), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1164))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
hist = model.fit_generator(train_gen, steps_per_epoch=EPOCH_STEPS, verbose=1, validation_data = valid_gen, validation_steps=VALID_STEPS, epochs=EPOCHS)

Epoch 1/5
3421/3421 [==============================] - 134s - loss: 0.0124 - val_loss: 0.0104
Epoch 2/5
3421/3421 [==============================] - 127s - loss: 0.0114 - val_loss: 0.0101
Epoch 3/5
3421/3421 [==============================] - 127s - loss: 0.0111 - val_loss: 0.0099
Epoch 4/5
3421/3421 [==============================] - 127s - loss: 0.0107 - val_loss: 0.0099
Epoch 5/5
3421/3421 [==============================] - 127s - loss: 0.0105 - val_loss: 0.0097


In [67]:
model.save('model.h5')